In [1]:
import sys
sys.path.append('../..')
sys.path.append('..')
from selective_recruitment.plotting import plot_parcels
from selective_recruitment.plotting import plot_connectivity_weight
from selective_recruitment.plotting import plot_mapwise_recruitment
from selective_recruitment.plotting import make_scatterplot
from selective_recruitment.scripts import script_get_stats_maps as sgs 

from selective_recruitment.scripts.script_mapwise import calc_ttest_mean

import selective_recruitment.regress as ra
import selective_recruitment.globals as gl
import selective_recruitment.data as ss
import selective_recruitment.region as sroi
import selective_recruitment.utils as sutil

import Functional_Fusion.atlas_map as am
import Functional_Fusion.dataset as ds


from statsmodels.stats.anova import AnovaRM  # perform F test
import statsmodels
from scipy import stats


from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import SUITPy.flatmap as flatmap
import nitools as nt
import nilearn.plotting as plotting
from nilearn import datasets # this will be used to plot sulci on top of the surface
import matplotlib.colors as colors
from matplotlib.colors import LinearSegmentedColormap
from nilearn.plotting.cm import _cmap_d as nilearn_cmaps

In [2]:
# setting working directory
wkdir = 'A:\data\Cerebellum\CerebellumWorkingMemory\selective_recruit'
if not Path(wkdir).exists():
    wkdir = '/srv/diedrichsen/data/Cerebellum/CerebellumWorkingMemory/selective_recruit'
if not Path(wkdir).exists():
    wkdir = '/Users/jdiedrichsen/Data/wm_cerebellum/selective_recruit'
if not Path(wkdir).exists():
    wkdir = '/Volumes/diedrichsen_data$/data/Cerebellum/CerebellumWorkingMemory/selective_recruit'

In [3]:
# setting defaults for plotting and analysis
dataset = "WMFS"
ses_id = "ses-02"
atlas_suit = "SUIT3"
atlas_fs = "fs32k"
# get atlas objects
atlas_cereb, atlas_info = am.get_atlas(atlas_str = atlas_suit, atlas_dir = gl.atlas_dir)
atlas_cortex, atlas_info = am.get_atlas(atlas_str = atlas_fs, atlas_dir = gl.atlas_dir)

# Which Regions from Glasser parcellation show significant activation during the task?

In [4]:
# create an instance of the dataset class
# cortex = "Icosahedron1002"
# # cortex = "Icosahedron1442"
cortex = "glasser"
# Data = ds.get_dataset_class(base_dir = gl.base_dir, dataset = "WMFS")
# # create the summary dataframe for glasser parcellation
# # get cortical data to be used as input to the connectivity model
# X,info,dset = ds.get_dataset(gl.base_dir,"WMFS",
#                                 atlas="fs32k", # for cortex we always use fs32k
#                                 sess="ses-02",
#                                 subj=None,
#                                 type = "CondAll")
# cortex_df = ss.get_summary_roi(X, info, 
#                                     atlas_space = "fs32k",
#                                     atlas_roi = cortex, 
#                                     unite_struct = False,
#                                     add_rest = True, 
#                                     var = "X")
# # save cortex_df as a pandas dataframe
# cortex_df.to_csv(Path(wkdir) / f"{cortex}_df.csv")
cortex_df = pd.read_csv(Path(wkdir) / f"{cortex}_df.csv", index_col = 0)

In [5]:
# limiting the statistical analysis to the multiple demand network
selected_regions = ["a9-46v", "p10p", "a10p", "11l", "a47r", 
                    "p47r", "FOP5", "AVl", "6r", "IFJp", "8C", "p9-46v", 
                    "i6-8", "s6-8", "AIP", "IP2", "IP1", 
                    "LIPd", "MIP", "PGs", "PFm", "TE1m", "TE1p", 
                    "POS2", "SCEF", "8BM", "a32pr", "d32"]

# print(len(selected_regions))

selected_names = [ f"{hemi}_{s}_ROI" for s in selected_regions for hemi in ["L", "R"]]

df = cortex_df.loc[np.isin(cortex_df["roi_name"], selected_names)]


In [6]:
# get inflated cortical surfaces
surfs = [gl.atlas_dir + f'/tpl-fs32k/tpl_fs32k_hemi-{h}_inflated.surf.gii' for i, h in enumerate(['L', 'R'])]
# plot tstat for encoding
surf_map_enc, sigs_enc = sgs.surf_stat(cortex_df, 
                      phase = 0, 
                      values = "X",
                      positive = True,
                      mult_comp = "hs",
                      cortex_roi = cortex, 
                      alpha = 0.01, 
                      corrected_map = False)


# plot tstat for retrieval
surf_map_ret, sigs_ret = sgs.surf_stat(cortex_df, 
                      phase = 1, 
                      values = "X",
                      positive = True,
                      mult_comp = "hs",
                      cortex_roi = cortex, 
                      alpha = 0.01, 
                      corrected_map = False)

Index(['L_31a_ROI', 'L_31pd_ROI', 'L_31pv_ROI', 'L_47m_ROI', 'L_47s_ROI',
       'L_4_ROI', 'L_52_ROI', 'L_55b_ROI', 'L_6a_ROI', 'L_6ma_ROI',
       ...
       'R_V3_ROI', 'R_V4_ROI', 'R_V4t_ROI', 'R_V8_ROI', 'R_VIP_ROI',
       'R_a24_ROI', 'R_d23ab_ROI', 'R_d32_ROI', 'R_p32pr_ROI', 'R_v23ab_ROI'],
      dtype='object', name='roi_name', length=101)
Index(['L_10d_ROI', 'L_10r_ROI', 'L_1_ROI', 'L_25_ROI', 'L_2_ROI',
       'L_31pd_ROI', 'L_31pv_ROI', 'L_47m_ROI', 'L_4_ROI', 'L_6a_ROI',
       ...
       'R_V3_ROI', 'R_V4_ROI', 'R_V4t_ROI', 'R_V8_ROI', 'R_VIP_ROI',
       'R_VVC_ROI', 'R_a24pr_ROI', 'R_d23ab_ROI', 'R_p32pr_ROI', 'R_s32_ROI'],
      dtype='object', name='roi_name', length=108)


In [24]:
# plot 
cmap = "cold_hot"
colorbar = False
stat_enc_fig = []
stat_ret_fig = []

# views   
for h,hemi in enumerate(['left', 'right']):
    
    for v in ['lateral', 'medial', 'anterior', 'posterior', 'ventral', 'dorsal']:
    
        print(f"view {v} hemi {hemi}")
        fig_enc = plotting.plot_surf_stat_map(
                            surfs[h], surf_map_enc[h], hemi=hemi,
                            colorbar=colorbar, 
                            view = v,
                            cmap=cmap,
                            engine='plotly',
                            symmetric_cbar = False,
                            vmax = 13, 
                            threshold = 5,
                        ).figure
        
        fig_ret = plotting.plot_surf_stat_map(
                            surfs[h], surf_map_ret[h], hemi=hemi,
                            colorbar=colorbar, 
                            view = v,
                            cmap=cmap,
                            engine='plotly',
                            symmetric_cbar = False,
                            vmax = 13, 
                            threshold = 5,
                        ).figure
        print(f"working_memory_stat_mult_comp_enc_{hemi}_{v}.pdf")
        fig_enc.write_image(f"working_memory_stat_mult_comp_enc_{hemi}_{v}.pdf")
        fig_ret.write_image(f"working_memory_stat_mult_comp_ret_{hemi}_{v}.pdf")

view lateral hemi left
working_memory_stat_mult_comp_enc_left_lateral.pdf
view medial hemi left
working_memory_stat_mult_comp_enc_left_medial.pdf
view anterior hemi left
working_memory_stat_mult_comp_enc_left_anterior.pdf
view posterior hemi left
working_memory_stat_mult_comp_enc_left_posterior.pdf
view ventral hemi left
working_memory_stat_mult_comp_enc_left_ventral.pdf
view dorsal hemi left
working_memory_stat_mult_comp_enc_left_dorsal.pdf
view lateral hemi right
working_memory_stat_mult_comp_enc_right_lateral.pdf
view medial hemi right
working_memory_stat_mult_comp_enc_right_medial.pdf
view anterior hemi right
working_memory_stat_mult_comp_enc_right_anterior.pdf
view posterior hemi right
working_memory_stat_mult_comp_enc_right_posterior.pdf
view ventral hemi right
working_memory_stat_mult_comp_enc_right_ventral.pdf
view dorsal hemi right
working_memory_stat_mult_comp_enc_right_dorsal.pdf


# Which Regions from hierarchical parcellation show significant activation during the task?

In [ ]:
# # create an instance of the dataset class
# Data = ds.get_dataset_class(base_dir = gl.base_dir, dataset = "WMFS")
# # create the summary dataframe for glasser parcellation
# # get cortical data to be used as input to the connectivity model
# Y,info,dset = ds.get_dataset(gl.base_dir,"WMFS",
#                                 atlas="SUIT3", # for cortex we always use fs32k
#                                 sess="ses-02",
#                                 subj=None,
#                                 type = "CondAll")
# cereb1_df = ss.get_summary_roi(Y, info, 
#                                     atlas_space = "SUIT3",
#                                     atlas_roi = "NettekovenSym32integLRAP", 
#                                     unite_struct = False,
#                                     add_rest = True, 
#                                     var = "Y")

# # cereb2_df = ss.get_summary_roi(Y, info, 
# #                                     atlas_space = "SUIT3",
# #                                     atlas_roi = "NettekovenSym32AP", 
# #                                     unite_struct = False,
# #                                     add_rest = True, 
# #                                     var = "Y")

# # # save cortex_df as a pandas dataframe
# cereb1_df.to_csv(Path(wkdir) / "NettekovenSym32integLRAP_df.csv")

# # save cortex_df as a pandas dataframe
# cereb2_df.to_csv(Path(wkdir) / "NettekovenSym32AP_df.csv")

cereb1_df = pd.read_csv(Path(wkdir) / "NettekovenSym32integLRAP_df.csv", index_col = 0)
# cereb2_df = pd.read_csv(Path(wkdir) / "NettekovenSym32AP_df.csv", index_col = 0)

In [ ]:
# get dataframe corresponding to regions whose name starts with "D
dd = cereb1_df[cereb1_df["roi_name"].str.startswith("I")]
dd.roi_name.unique()

In [ ]:

flat_enc = sgs.flatmap_stat(cereb1_df, 
             phase = 0, 
             values = "Y",
             mult_comp = "hs",
             alpha = 0.01)
print(flat_enc)

# # flat_enc
# enc_fig = flatmap.plot(data=flat_enc, 
#                             render="plotly", 
#                             overlay_type="func",
#                             hover='auto', 
#                             cmap = cmap, 
#                             colorbar = False, 
#                             bordersize = 1.5,
#                             bordercolor = "black",  
#                             cscale=[2, 7]
#                             )

# enc_fig.show()